In [1]:
import os
from dotenv import load_dotenv
from IPython.display import display, Image

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.tools.retriever import create_retriever_tool
from langgraph.checkpoint.memory import InMemorySaver

from typing import List
import logging
from langchain_community.tools import BraveSearch
from langchain_tavily import TavilyExtract
import json

# For QA-Retriever
from langchain_core.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Custom Import
from optcg.vectorstore_logic import check_for_updates_to_rules, create_or_load_vectorstore_optcg_rulebooks
from optcg.agents import RulebookAgent
from optcg.tools import web_search_tool, youtube_search_tool

from langsmith import traceable
from langsmith.wrappers import wrap_openai
from openai import OpenAI
#openai_client = wrap_openai(OpenAI())

_ = load_dotenv() # Loads the .env file - e.g. the OPENAI_API_KEY

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


In [2]:
from optcg.tools import web_search_tool
type(web_search_tool), web_search_tool.invoke("What is the power of a card in the One Piece TCG?")

(langchain_core.tools.structured.StructuredTool,
 [{'url': 'https://www.dicebreaker.com/games/one-piece-card-game/how-to/how-to-play-one-piece-card-game',
   'raw_content': 'Published Time: 2023-07-26T15:48:50.574345+00:00\n\nHow to Play the One Piece Card Game | Dicebreaker\n\n===============\n\n[Skip to main content](https://www.dicebreaker.com/games/one-piece-card-game/how-to/how-to-play-one-piece-card-game#main-content)\n*   [Most expensive Pokémon card](https://www.dicebreaker.com/games/pokemon-trading-card-game/best-games/rare-pokemon-cards)\n*   [Best board games](https://www.dicebreaker.com/categories/board-game/best-games/best-board-games-2024)\n*   [MTG Arena codes](https://www.dicebreaker.com/games/magic-the-gathering-game/how-to/mtg-arena-codes)\n*   [Yu-Gi-Oh! Banlist](https://www.dicebreaker.com/games/yu-gi-oh-tcg/how-to/yu-gi-oh-banlist-forbidden-limited)\n*   [Best Lorcana starter deck](https://www.dicebreaker.com/games/disney-lorcana/best-games/best-lorcana-starter-dec

In [3]:
type(youtube_search_tool), youtube_search_tool.invoke("What is the power of a card in the One Piece TCG?")

(langchain_core.tools.structured.StructuredTool,
 [{'url': 'https://www.youtube.com/watch?v=DKqLcsXUYik&pp=ygUxV2hhdCBpcyB0aGUgcG93ZXIgb2YgYSBjYXJkIGluIHRoZSBPbmUgUGllY2UgVENHPw%3D%3D',
   'raw_content': "# How to Play the One Piece Card Game (TCG)\n## TrueChampionSteven\n27700 subscribers\n7536 likes\n\n### Description\n391130 views\nPosted: 1 Oct 2022\nIn this video, I will teach you how to play the One Piece Card Game.  The One Piece TCG is the latest collectible card game created by Bandai.  Popular in Japan, and getting released later this year worldwide, this game lets you battle and create your own crew of popular One Piece characters to hopefully become King of the Pirates.\n\n----------------------------------------------------------------------------------\n\nWATCH ME LIVE ►► https://www.twitch.tv/truechampionsteven\nfollow me on twitter ►► https://twitter.com/championsteven_\njoin my discord ►► https://discord.gg/kNmkgH2\n\nOfficial Website ►► https://en.onepiece-cardgame.co